In [5]:
from stable_baselines3 import SAC
import gymnasium as gym

# Create environment and model
env = gym.make("Pendulum-v1")  # This has a continuous action space
model = SAC("MlpPolicy", env, verbose=1)

policy_network = model.policy
q_networks = model.critic  # This contains both Q networks

# Access the first and second Q networks
q0_net = model.critic.qf0
q2_net = model.critic_target.qf0
print(policy_network)
# print(q0_net)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
SACPolicy(
  (actor): Actor(
    (features_extractor): FlattenExtractor(
      (flatten): Flatten(start_dim=1, end_dim=-1)
    )
    (latent_pi): Sequential(
      (0): Linear(in_features=3, out_features=256, bias=True)
      (1): ReLU()
      (2): Linear(in_features=256, out_features=256, bias=True)
      (3): ReLU()
    )
    (mu): Linear(in_features=256, out_features=1, bias=True)
    (log_std): Linear(in_features=256, out_features=1, bias=True)
  )
  (critic): ContinuousCritic(
    (features_extractor): FlattenExtractor(
      (flatten): Flatten(start_dim=1, end_dim=-1)
    )
    (qf0): Sequential(
      (0): Linear(in_features=4, out_features=256, bias=True)
      (1): ReLU()
      (2): Linear(in_features=256, out_features=256, bias=True)
      (3): ReLU()
      (4): Linear(in_features=256, out_features=1, bias=True)
    )
    (qf1): Sequential(
      (0): Linear(in_features=4, out_featur

In [6]:
def dummy_process(shared_array):
    critic_net = shared_array[0]
    # env = gym.make("Pendulum-v1")  # This has a continuous action space
    print("Critic's addr in child process", id(critic_net))
    # target_agent.run_episode()

In [7]:
from SAC_agent_package.prime_model import prime_SAC_model
from custom_utils import prepare_agent
import torch.multiprocessing as tmp
env = gym.make("Pendulum-v1")  # This has a continuous action space
tmp.set_start_method('spawn',force=True) 

# model = prime_SAC_model("MlpPolicy", env, verbose=1)
keyargs = {
    'policy' : "MlpPolicy",
    'env' : env,
    'verbose': 1
}

target_agent = prepare_agent(prime_SAC_model, keyargs)
# model: prime_SAC_model = target_agent()
# model.share_model_across_memory()



env = gym.make("Blackjack-v1", sab=False)

def dummy_process(target_agent):
    # print("Critic's",id(target_agent.critic))
    model: prime_SAC_model = target_agent()
    model.run_episode()
    # print("Running child")
# print(BJA.BlackjackAgent.__init__)


child_process = tmp.Process(target=dummy_process, kwargs={"target_agent": target_agent})

child_process.start()

child_process.join(10)
# print(model.run_episode())

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/opt/anaconda3/envs/GNC_Project_Env/lib/python3.9/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/opt/anaconda3/envs/GNC_Project_Env/lib/python3.9/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'dummy_process' on <module '__main__' (built-in)>


In [8]:
from stable_baselines3 import SAC
from stable_baselines3.common.evaluation import evaluate_policy
import gymnasium as gym

# Create environment and model
env = gym.make("Pendulum-v1")  # This has a continuous action space
model = SAC("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=1000)
mean_reward , std_reward = evaluate_policy(model, env,n_eval_episodes=10)

print(f"Linear trained model: Mean reward: {mean_reward}, Std reward: {std_reward}")

mp_model = SAC("MlpPolicy", env, verbose=1)
mp_model.load("./output_models/test_model.zip")
mean_reward , std_reward = evaluate_policy(mp_model, env,n_eval_episodes=10)
print(f"Mp trained model: Mean reward: {mean_reward}, Std reward: {std_reward}")

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 200       |
|    ep_rew_mean     | -1.37e+03 |
| time/              |           |
|    episodes        | 4         |
|    fps             | 190       |
|    time_elapsed    | 4         |
|    total_timesteps | 800       |
| train/             |           |
|    actor_loss      | 21.7      |
|    critic_loss     | 0.213     |
|    ent_coef        | 0.812     |
|    ent_coef_loss   | -0.344    |
|    learning_rate   | 0.0003    |
|    n_updates       | 699       |
----------------------------------
Linear trained model: Mean reward: -1548.7864421136678, Std reward: 217.21333169464916
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Mp trained model: Mean reward: -1399.514025856927, Std reward: 293.43736190188196
